In [1]:
# preprocess
import preprocess
preprocess.word_segment('process/train_data_remove_spell.txt','process/train_data_remove_spell_words.csv')
preprocess.word_segment('process/test_data_remove_spell.txt','process/test_data_remove_spell_words.csv')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\h_hg\AppData\Local\Temp\jieba.cache
Loading model cost 2.190 seconds.
Prefix dict has been built succesfully.


In [2]:
# load training data, testing data
import preprocess
X = preprocess.load_word_segment('process/train_data_remove_spell_words.csv')
X_verify = preprocess.load_word_segment('process/test_data_remove_spell_words.csv')

def load_one_label(input_path):
    labels = None
    with open(input_path, mode='r',encoding='utf-8') as f:
        labels = [int(label.strip()) for label in f.readlines()]
    return labels

Y = load_one_label('process/train_labels.txt')

In [3]:
import pickle
def save_model(file_path, model):
    with open(file_path, 'wb') as f:
        pickle.dump(model, f)

def load_model(file_path):
    model = None
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
    return model

In [4]:
def get_model_name(prefix, params):
    name = prefix
    for param in sorted(params.items(), key=lambda pair:pair[0]):
        name += "_" + param[0] + "_" + str(param[1])
    name += ".bin"
    return name

In [5]:
# word2vec
import os
import time
from gensim.models import Word2Vec

wv_params = {"iter":10, "min_count":20,"size":20}

model_path = "bin/" + get_model_name('wv',wv_params)

start_time = time.time()

if os.path.exists(model_path):
    word2vec_model = load_model(model_path)
else:
    word2vec_model = Word2Vec(X, **wv_params)
    save_model(model_path, word2vec_model)

used_time = time.time() - start_time
print(u'Initialization of building word2vec model: %.3fs' %used_time)

Initialization of building word2vec model: 2.356s


In [6]:
import numpy as np

def words_to_vector(words, word2vec_model):
    vector_list = [ word2vec_model.wv[word] for word in words if word in word2vec_model]
    return np.array(vector_list).mean(axis=0)

def word2vec(words_list, word2vec_model):
    vec_list = [ words_to_vector(words,word2vec_model) for words in words_list]
    """
    vec_list = []
    for i in range(len(words_list)):
        vec = words_to_vector(words_list[i],word2vec_model)
        if str(vec.shape) != '(20,)': # 20要改成wv_params里面的参数
            print(i+1, words_list[i])
        vec_list.append(vec)
    """
    return np.array(vec_list)

In [7]:
wv_X = word2vec(X, word2vec_model)
print(wv_X.shape)
wv_X_verify = word2vec(X_verify, word2vec_model)
print(wv_X_verify.shape)

C:\ProgramData\scoop\apps\python\current\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
(10000, 20)
(2000, 20)


In [10]:
from sklearn.metrics import confusion_matrix
def report(y_test, y_pred):
    n = np.size(y_test)
    y_test = np.array(y_test).reshape((n))
    y_pred = np.array(y_pred).reshape((n))
    C = confusion_matrix(y_test, y_pred)
    TP = C[0][0]
    TN = C[0][1]
    FP = C[1][0]
    FN = C[1][1]

    print("acc: ", np.sum(y_test == y_pred)/n)
    print("confusion_matrix\n", C)
    
    print("safe")
    P = 1.0 * TP / (FP + TP)
    R = 1.0 * TP / (TP + FN)
    F1 = 2.0 * P * R / (P + R)
    print("P: ", P)
    print("R: ", R)
    print("F1: ", F1)

    print("unsafe")
    P = 1.0 * TN / (TN + FN)
    R = 1.0 * TN / (TN + FP)
    F1 = 2.0 * P * R / (P + R)
    print("P: ", P)
    print("R: ", R)
    print("F1: ", F1)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(wv_X, Y, test_size=0.2)


acc:  0.9215
confusion_matrix
 [[1655   55]
 [ 102  188]]
non-fraud
P:  0.941946499715424
R:  0.8979924036896365
F1:  0.9194444444444444
fraud
P:  0.22633744855967078
R:  0.3503184713375796
F1:  0.27499999999999997
C:\ProgramData\scoop\apps\python\current\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
# logistic Regression
logisticRegression_model = LogisticRegression()
logisticRegression_model.fit(x_train, y_train)
y_pred = logisticRegression_model.predict(x_test)
report(y_test, y_pred)